In [ ]:
import pandas as pd    # to load dataset
import numpy as np     # for mathematic equation
from nltk.corpus import stopwords   # to get collection of stopwords
from sklearn.model_selection import train_test_split       # for splitting dataset
from tensorflow.keras.preprocessing.text import Tokenizer  # to encode text to int
from tensorflow.keras.preprocessing.sequence import pad_sequences   # to do padding or truncating
from tensorflow.keras.models import Sequential     # the model
from tensorflow.keras.layers import Embedding, LSTM, Dense # layers of the architecture
from tensorflow.keras.callbacks import ModelCheckpoint   # save model
from tensorflow.keras.models import load_model   # load saved model
import re

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

# **Loading and cleaning dataset**

In [ ]:
english_stops = set(stopwords.words('english'))
def load_dataset():
    df = pd.read_csv('IMDB Dataset.csv')
    x_data = df['review']       # Reviews/Input
    y_data = df['sentiment']    # Sentiment/Output

    # PRE-PROCESS REVIEW
    x_data = x_data.replace({'<.*?>': ''}, regex = True)          # remove html tag
    x_data = x_data.replace({'[^A-Za-z]': ' '}, regex = True)     # remove non alphabet
    x_data = x_data.apply(lambda review: [w for w in review.split() if w not in english_stops])  # remove stop words
    x_data = x_data.apply(lambda review: [w.lower() for w in review])   # lower case
    
    # ENCODE SENTIMENT -> 0 & 1
    y_data = y_data.replace('positive', 1)
    y_data = y_data.replace('negative', 0)

    return x_data, y_data

x_data, y_data = load_dataset()


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.5)
def get_max_length():
    review_length = []
    for review in x_train:
        review_length.append(len(review))

    return int(np.ceil(np.mean(review_length)))

In [ ]:
x_train.shape,x_test.shape

((25000, 129), (25000, 129))

# Tokenize and Pad/Truncate Reviews

A Neural Network only accepts numeric data, so we need to encode the reviews. I use tensorflow.keras.preprocessing.text.Tokenizer to encode the reviews into integers, where each unique word is automatically indexed (using fit_on_texts method) based on x_train.
x_train and x_test is converted into integers using texts_to_sequences method.

Each reviews has a different length, so we need to add padding (by adding 0) or truncating the words to the same length (in this case, it is the mean of all reviews length) using tensorflow.keras.preprocessing.sequence.pad_sequences.

post- pad or truncate the words in the back of a sentence

pre- pad or truncate the words in front of a sentence

In [ ]:
# ENCODE REVIEW
token = Tokenizer(lower=False)    # no need lower, because already lowered the data in load_data()
token.fit_on_texts(x_train)
x_train = token.texts_to_sequences(x_train)
x_test = token.texts_to_sequences(x_test)

max_length = get_max_length()

x_train = pad_sequences(x_train, maxlen=max_length, padding='post', truncating='post')
x_test = pad_sequences(x_test, maxlen=max_length, padding='post', truncating='post')

total_words = len(token.word_index) + 1   # add 1 because of 0 padding

# Build Architecture/Model
Embedding Layer: in simple terms, it creates word vectors of each word in the word_index and group words that are related or have similar meaning by analyzing other words around them.

LSTM Layer: to make a decision to keep or throw away data by considering the current input, previous output, and previous memory. There are some important components in LSTM.

Forget Gate- decides information is to be kept or thrown away

Input Gate- updates cell state by passing previous output and current input into sigmoid activation function

Cell State- calculate new cell state, it is multiplied by forget vector (drop value if multiplied by a near 0), add it with the output from input gate to update the cell state value.

Ouput Gate- decides the next hidden state and used for predictions

Dense Layer: compute the input with the weight matrix and bias (optional), and using an activation function. I use Sigmoid activation function for this work because the output is only 0 or 1.

The optimizer is Adam and the loss function is Binary Crossentropy because again the output is only 0 and 1, which is a binary number

In [ ]:
# ARCHITECTURE
EMBED_DIM = 32
LSTM_OUT = 64

model = Sequential()
model.add(Embedding(total_words, EMBED_DIM, input_length = max_length))
model.add(LSTM(LSTM_OUT))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 129, 32)           2417280   
_________________________________________________________________
lstm (LSTM)                  (None, 64)                24832     
_________________________________________________________________
dense (Dense)                (None, 1)                 65        
Total params: 2,442,177
Trainable params: 2,442,177
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
checkpoint = ModelCheckpoint(
    'models/LSTM.h5',
    monitor='accuracy',
    save_best_only=True,
    verbose=1
)
model.fit(x_train, y_train, batch_size = 128, epochs = 5, callbacks=[checkpoint])


Epoch 1/5
196/196 [==============================] - 40s 191ms/step - loss: 0.5228 - accuracy: 0.7029

Epoch 00001: accuracy improved from -inf to 0.70292, saving model to models/LSTM.h5
Epoch 2/5
196/196 [==============================] - 37s 191ms/step - loss: 0.2367 - accuracy: 0.9165

Epoch 00002: accuracy improved from 0.70292 to 0.91648, saving model to models/LSTM.h5
Epoch 3/5
196/196 [==============================] - 37s 188ms/step - loss: 0.1279 - accuracy: 0.9618

Epoch 00003: accuracy improved from 0.91648 to 0.96180, saving model to models/LSTM.h5
Epoch 4/5
196/196 [==============================] - 37s 189ms/step - loss: 0.0798 - accuracy: 0.9801

Epoch 00004: accuracy improved from 0.96180 to 0.98008, saving model to models/LSTM.h5
Epoch 5/5
196/196 [==============================] - 37s 188ms/step - loss: 0.0532 - accuracy: 0.9885

Epoch 00005: accuracy improved from 0.98008 to 0.98852, saving model to models/LSTM.h5


In [ ]:
results = model.fit(
 x_train, y_train,
 epochs= 5,
 batch_size = 100,
 validation_data = (x_test, y_test)
)

Epoch 1/5
250/250 [==============================] - 47s 182ms/step - loss: 0.0605 - accuracy: 0.9864 - val_loss: 0.5259 - val_accuracy: 0.8605
Epoch 2/5
250/250 [==============================] - 45s 182ms/step - loss: 0.0566 - accuracy: 0.9861 - val_loss: 0.5612 - val_accuracy: 0.8563
Epoch 3/5
250/250 [==============================] - 46s 183ms/step - loss: 0.0436 - accuracy: 0.9898 - val_loss: 0.5907 - val_accuracy: 0.8482
Epoch 4/5
250/250 [==============================] - 45s 182ms/step - loss: 0.0330 - accuracy: 0.9927 - val_loss: 0.6074 - val_accuracy: 0.8566
Epoch 5/5
250/250 [==============================] - 45s 180ms/step - loss: 0.0286 - accuracy: 0.9944 - val_loss: 0.6934 - val_accuracy: 0.8475


In [ ]:
y_pred = model.predict_classes(x_test, batch_size = 128)
#y_pred=np.argmax(model.predict(x_test, batch_size = 128), axis=-1)

true = 0
for i, y in enumerate(y_test):
    if y == y_pred[i]:
        true += 1

print('Correct Prediction: {}'.format(true))
print('Wrong Prediction: {}'.format(len(y_pred) - true))
print('Accuracy: {}'.format(true/len(y_pred)*100))

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


Correct Prediction: 21444
Wrong Prediction: 3556
Accuracy: 85.776
